In [59]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="quick-starts-ws-131689")
ws = Workspace.get(name="AML-Alex")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: AML-Alex
Azure region: eastus2
Subscription id: f56abefd-7248-4d0f-a3b1-3b23fcdb0bc7
Resource group: AzureML


In [60]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes= 0,
                                                           max_nodes=4, vm_priority='lowpriority')
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

CreatingAmlCompute is getting created. Consider calling wait_for_completion() first

AmlCompute is getting created. Consider calling wait_for_completion() first

AmlCompute is getting created. Consider calling wait_for_completion() first

AmlCompute is getting created. Consider calling wait_for_completion() first

AmlCompute is getting created. Consider calling wait_for_completion() first

AmlCompute is getting created. Consider calling wait_for_completion() first


Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [84]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "C": uniform(0.0001,0.1),
        "max_iter": choice(100,500,1000)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor= 0.1, evaluation_interval = 1, delay_evaluation=5)

# if "training" not in os.listdir():
#     os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = '.', compute_target = cpu_cluster_name  , entry_script='train.py')   

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=2,
                                     max_concurrent_runs=4)

In [85]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [96]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_metrics())
# joblib.dump(model,'model.joblib')

best_run.get_file_names()

best_run.download_file('outputs/'+best_run.id+'_model.joblib')

{'Regularization Strength:': 0.032319768229719804, 'Max iterations:': 500, 'Accuracy': 0.9097622660596864}


In [92]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

ds = TabularDatasetFactory.from_delimited_files(path,separator=',', header=True)


In [89]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
0,57,1,0,0,1,5,1,371,1,999,...,1,0,0,0,0,1,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,1,0,0,0,0,0,0,0,1
2,33,1,0,0,0,5,5,52,1,999,...,1,0,0,0,1,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,0,1,0,0,0,1,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,1,0,0,0,0,1,0,0,0,0


In [100]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task= 'classification',
    primary_metric='accuracy',
    training_data=x,
    label_column_name='y',
    n_cross_validations=5)

In [101]:
# Submit your automl run

run = exp.submit(automl_config, show_output = True)
run.wait_for_completion()

Running on local machine


ValidationException: ValidationException:
	Message: Please install specific versions of packages: azureml-defaults requires flask==1.0.3 but has Flask 1.1.2.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Please install specific versions of packages: azureml-defaults requires flask==1.0.3 but has Flask 1.1.2.",
        "target": "azureml-defaults",
        "inner_error": {
            "code": "NotSupported",
            "inner_error": {
                "code": "IncompatibleOrMissingDependency"
            }
        },
        "reference_code": "435ab938-fd87-49bc-932e-6eec0d6aee4f"
    }
}

In [ ]:
# Retrieve and save your best automl model.

best_run_customized, fitted_model_customized = run.get_output()

In [ ]:
# cluster clean up

cpu_cluster.delete()
print(cpu_cluster)